# Generalization with OOP: The ConBacktester Class

## Using the ConBacktester Class

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [ ]:
df = pd.read_csv("intraday_pairs.csv", parse_dates = ["time"], index_col = "time")
df

In [ ]:
df.info()

In [ ]:
ptc = 0.00007

In [ ]:
import ConBacktester as Con

In [ ]:
tester = Con.ConBacktester("EURUSD", "2018-01-01", "2019-12-31", ptc)

In [ ]:
tester

In [ ]:
tester.test_strategy()

In [ ]:
tester.plot_results()

In [ ]:
tester.results

In [ ]:
tester.optimize_parameter((1, 500, 1))

In [ ]:
tester.plot_results()

In [ ]:
tester.results_overview

In [ ]:
tester.results_overview.nlargest(10, "performance")

In [ ]:
tester.results_overview.nsmallest(10, "performance")

In [ ]:
tester = Con.ConBacktester("GBPUSD", "2018-01-01", "2019-12-31", 0)

In [ ]:
tester.test_strategy()

In [ ]:
tester.plot_results()

In [ ]:
tester.results

In [ ]:
tester.results.trades.value_counts()

In [ ]:
tester.optimize_parameter((1, 500, 1))

In [ ]:
tester.plot_results()

In [ ]:
tester.results.trades.value_counts()

## OOP Challenge: Create the ConBacktester Class from scratch

__Steps__:

1. Initialize the properties __symbol__, __start__, __end__, __tc__, __results__. 

2. Add the method __get_data()__ retrieving & preparing raw data from "intraday_pairs.csv". Call get_data() in the "dunder init" method.

3. Add the method __test_strategy()__ with the paramter window = .

4. Add the method __plot_results()__.

5. Add the method __optimize_parameter__ (only one parameter, no combinations required)

6. Add an appropriate (string) __representation__ and __Docstrings__.

# Solutions (Stop here if you want to code on your own!)

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
class ConBacktester():
    ''' Class for the vectorized backtesting of simple contrarian trading strategies.
    '''    
    
    def __init__(self, symbol, start, end, tc):
        '''
        Parameters
        ----------
        symbol: str
            ticker symbol (instrument) to be backtested
        start: str
            start date for data import
        end: str
            end date for data import
        tc: float
            proportional transaction/trading costs per trade
        '''
        self.symbol = symbol
        self.start = start
        self.end = end
        self.tc = tc
        self.results = None
        self.get_data()
        
    def __repr__(self):
        return "ConBacktester(symbol = {}, start = {}, end = {})".format(self.symbol, self.start, self.end)
        
    def get_data(self):
        ''' Imports the data from intraday_pairs.csv (source can be changed).
        '''
        raw = pd.read_csv("intraday_pairs.csv", parse_dates = ["time"], index_col = "time")
        raw = raw[self.symbol].to_frame().dropna()
        raw = raw.loc[self.start:self.end].copy()
        raw.rename(columns={self.symbol: "price"}, inplace=True)
        raw["returns"] = np.log(raw / raw.shift(1))
        self.data = raw
        
    def test_strategy(self, window = 1):
        ''' Backtests the simple contrarian trading strategy.
        
        Parameters
        ----------
        window: int
            time window (number of bars) to be considered for the strategy.
        '''
        self.window = window
        data = self.data.copy().dropna()
        data["position"] = -np.sign(data["returns"].rolling(self.window).mean())
        data["strategy"] = data["position"].shift(1) * data["returns"]
        data.dropna(inplace=True)
        
        # determine the number of trades in each bar
        data["trades"] = data.position.diff().fillna(0).abs()
        
        # subtract transaction/trading costs from pre-cost return
        data.strategy = data.strategy - data.trades * self.tc
        
        data["creturns"] = data["returns"].cumsum().apply(np.exp)
        data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
        self.results = data
        
        perf = data["cstrategy"].iloc[-1] # absolute performance of the strategy
        outperf = perf - data["creturns"].iloc[-1] # out-/underperformance of strategy
        
        return round(perf, 6), round(outperf, 6)
    
    def plot_results(self):
        ''' Plots the performance of the trading strategy and compares to "buy and hold".
        '''
        if self.results is None:
            print("Run test_strategy() first.")
        else:
            title = "{} | Window = {} | TC = {}".format(self.symbol, self.window, self.tc)
            self.results[["creturns", "cstrategy"]].plot(title=title, figsize=(12, 8))
            
    def optimize_parameter(self, window_range):
        ''' Finds the optimal strategy (global maximum) given the window parameter range.

        Parameters
        ----------
        window_range: tuple
            tuples of the form (start, end, step size)
        '''
        
        windows = range(*window_range)
            
        results = []
        for window in windows:
            results.append(self.test_strategy(window)[0])
        
        best_perf = np.max(results) # best performance
        opt = windows[np.argmax(results)] # optimal parameter
        
        # run/set the optimal strategy
        self.test_strategy(opt)
        
        # create a df with many results
        many_results =  pd.DataFrame(data = {"window": windows, "performance": results})
        self.results_overview = many_results
        
        return opt, best_perf
                               